In [29]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [30]:
gis = gpd.read_file('../data/bundle/Eskom/MTS_Subs2022.shp').to_crs('EPSG:4326')
gis.index = gis['Substation']

In [31]:
lines = gpd.read_file('../data/bundle/Eskom/Planned_Lines.shp').to_crs('EPSG:2059')

In [32]:
def haversine_length(row):
    
    bus0 = gis.loc[row["bus0"], "geometry"]
    bus1 = gis.loc[row["bus1"], "geometry"]

    lon1, lat1, lon2, lat2 = map(np.radians, [bus0.x, bus0.y, bus1.x, bus1.y])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return c * 6371

In [33]:
mts = pd.Series({
    "Upington":"Aries",
    "Ferrum":"Upington",
    "Hydra":"Kronos",
    "Kronos":"Aries",
    "Beta":"Boundary",
    "Boundary":"Ferrum",
    "Gromis":"Nama",
    "Hermes":"Mookodi", 
    "Mookodi":"Ferrum",
    #"Aggeneis":"Groeipunt",
    "Aries":"Aggeneis",
    "Paulputs":"Aggeneis",
    #"Droerivier":"Narina" ,
    #"Narina":"Gourikwa",
    "Aurora":"Juno",
}).reset_index()
mts.columns = ["bus0", "bus1"]

In [34]:
gis = gis.loc[~gis.index.duplicated(keep='first')]

In [35]:
mts["distance"] = mts.apply(haversine_length, axis=1)

In [36]:
mts

,bus0,bus1,distance
0,Upington,Aries,110.730462
1,Ferrum,Upington,209.241691
2,Hydra,Kronos,184.700244
3,Kronos,Aries,160.227489
4,Beta,Boundary,80.259073
5,Boundary,Ferrum,209.972632
6,Gromis,Nama,68.108567
7,Hermes,Mookodi,202.585063
8,Mookodi,Ferrum,184.886282
9,Aries,Aggeneis,194.075686


In [37]:
lines["length"] = lines.length/1000

In [38]:
lines.to_csv("lines.csv")